In [8]:
import pandas as pd
from sklearn.utils import resample

real_data = pd.read_csv(r"C:\Users\Abhilash\Documents\python\motor\Production_data_montari_for_ml_model.csv") 

synthetic_data = resample(
    real_data,
    replace=True,
    n_samples=200,
    random_state=42
)

synthetic_data.to_csv("synthetic_data.csv", index=False)


In [2]:
!pip install sdv

   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ------- -------------------------------- 2.9/14.6 MB 15.2 MB/s eta 0:00:01
   --------------- ------------------------ 5.8/14.6 MB 14.1 MB/s eta 0:00:01
   ----------------------- ---------------- 8.4/14.6 MB 13.7 MB/s eta 0:00:01
   ------------------------------ --------- 11.3/14.6 MB 13.5 MB/s eta 0:00:01
   -------------------------------------- - 14.2/14.6 MB 13.5 MB/s eta 0:00:01
   ---------------------------------------- 14.6/14.6 MB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ----------- ---------------------------- 2.9/9.9 MB 12.9 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.9 MB 13.0 MB/s eta 0:00:01
   --------------------------------- ------ 8.4/9.9 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ----------

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xg
from xgboost import XGBRegressor 
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

df = pd.read_csv(r'C:\Users\Abhilash\Documents\python\motor\synthetic_data.csv')
print(df.head())


   Plan_Capacity  Man_power  SHIFT  Total_Down_time  Material_Loss  \
0            237         14    1.0                0              0   
1             13          5    0.5              225            225   
2             20          0    0.0               20              0   
3             23          2    0.5                0              0   
4            193         14    1.0               30              0   

   Has_downtime  Material_Issue  Prev_Day_Completion  Prev_Day_Downtime  \
0             0               0                100.0                  0   
1             1               1                100.0                  0   
2             1               0                100.0                  0   
3             0               0                 83.8                  0   
4             1               0                100.0                  0   

   Actual_Qty  
0         237  
1          13  
2          19  
3          23  
4         175  


In [40]:

np.random.seed(42)
df['noise'] = np.random.normal(0, 0.05, len(df))  # 5% random variation
df['Actual_Qty'] = df['Actual_Qty'] * (1 + df['noise'])
df = df.drop('noise', axis=1)

In [41]:

X = df[['Plan_Capacity', 'Man_power', 'SHIFT', 'Total_Down_time', 
        'Material_Loss', 'Has_downtime', 'Material_Issue', 
        'Prev_Day_Completion', 'Prev_Day_Downtime']]
y = df['Actual_Qty']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:

print(f"Training{X_train.shape}, Testing{X_test.shape}")

Training(160, 9), Testing(40, 9)


In [44]:
xgb_model = xg.XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=1,
    reg_alpha=0.1,     
    reg_lambda=1.0,      
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train, 
              eval_set=[(X_test, y_test)],
               verbose=False)

y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)


train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Train MAE: {train_mae:.1f} units")
print(f"Test MAE:  {test_mae:.1f} units")
print(f"Train R²:  {train_r2:.3f}")
print(f"Test R²:   {test_r2:.3f}")

Train MAE: 2.7 units
Test MAE:  3.9 units
Train R²:  0.997
Test R²:   0.995


In [ ]:
import joblib
import pickle

joblib.dump(xgb_model, 'production_xgboost_model.pkl')

joblib.dump(scaler, 'production_scaler.pkl')

feature_names = ['Plan_Capacity', 'Man_power', 'SHIFT', 'Total_Down_time', 
                'Material_Loss', 'Has_downtime', 'Material_Issue', 
                'Prev_Day_Completion', 'Prev_Day_Downtime']
with open('feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)



✅ SAVED PRODUCTION FILES:
- production_xgboost_model.pkl
- production_scaler.pkl
- feature_names.pkl


In [50]:
!pip install streamlit joblib pandas scikit-learn xgboost plotly

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   ------------ --------------------------- 2.9/9.1 MB 15.2 MB/s eta 0:00:01
   ------------------------ --------------- 5.5/9.1 MB 14.0 MB/s eta 0:00:01
   ------------------------------------ --- 8.4/9.1 MB 13.7 MB/s eta 0:00:01
   ---------------------------------------- 9.1/9.1 MB 13.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/795.4 kB ? eta -:--:--
   --------------------------------------- 795.4/795.4 kB 16.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------- ----------------------- 2.9/6.9 MB 14.0 MB/s eta 0:00:01
   --------------------------------- ------ 5.8/6.9 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------- 6.9/6.9 MB 13.3 MB/s eta 0:00:00
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
   ---------------------------------------- 0.0/28.1 M